In [4]:
import cv2
import numpy as np
import json
import os
from ultralytics import YOLO
from datetime import datetime
from tqdm import tqdm

def extract_coords_from_filename(filename):
    """Извлекает координаты из названия файла"""
    # Пример: map_55d954635_37d942221_to_55d962248_37d953572_z18.jpg
    try:
        base_name = os.path.splitext(filename)[0]  # убираем расширение
        parts = base_name.split('_')
        
        # Извлекаем координаты и заменяем 'd' обратно на точки
        lat1 = float(parts[1].replace('d', '.'))
        lon1 = float(parts[2].replace('d', '.'))
        lat2 = float(parts[4].replace('d', '.'))
        lon2 = float(parts[5].replace('d', '.'))
        
        # ОПРЕДЕЛЯЕМ СЕВЕРНУЮ И ЮЖНУЮ ГРАНИЦЫ
        # Широта: северная больше, южная меньше
        north_lat = max(lat1, lat2)
        south_lat = min(lat1, lat2)
        
        # Долгота: западная меньше, восточная больше  
        west_lon = min(lon1, lon2)
        east_lon = max(lon1, lon2)
        
        top_left_gps = (north_lat, west_lon)     # Северо-запад (северная широта, западная долгота)
        bottom_right_gps = (south_lat, east_lon) # Юго-восток (южная широта, восточная долгота)
        
        print(f"🔍 Извлеченные координаты из имени файла:")
        print(f"   Точка 1: {lat1}, {lon1}")
        print(f"   Точка 2: {lat2}, {lon2}")
        print(f"   Северо-запад (top_left): {top_left_gps}")
        print(f"   Юго-восток (bottom_right): {bottom_right_gps}")
        
        return top_left_gps, bottom_right_gps
        
    except Exception as e:
        print(f"❌ Ошибка извлечения координат из имени файла: {e}")
        return None, None


def calculate_centroid(mask):
    """Вычисляет центроид бинарной маски"""
    try:
        moments = cv2.moments(mask)
        if moments["m00"] != 0:
            centroid_x = int(moments["m10"] / moments["m00"])
            centroid_y = int(moments["m01"] / moments["m00"])
            return (centroid_x, centroid_y)
        else:
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            if contours:
                x, y, w, h = cv2.boundingRect(contours[0])
                return (x + w//2, y + h//2)
            else:
                return None
    except Exception as e:
        print(f"Ошибка при вычислении центроида: {e}")
        return None

def pixel_to_gps(pixel_coords, image_width, image_height, top_left_gps, bottom_right_gps):
    """Конвертирует пиксельные координаты в GPS координаты"""
    x_px, y_px = pixel_coords
    
    # ПРАВИЛЬНЫЙ РАСЧЕТ ДИАПАЗОНОВ
    lat_range = top_left_gps[0] - bottom_right_gps[0]  # разница по широте
    lon_range = bottom_right_gps[1] - top_left_gps[1]  # разница по долготе
    
    # Нормализация координат (0-1)
    x_norm = x_px / (image_width - 1)  # делим на (width-1) для правильной интерполяции
    y_norm = y_px / (image_height - 1)  # делим на (height-1) для правильной интерполяции
    
    # ПРАВИЛЬНАЯ ИНТЕРПОЛЯЦИЯ:
    # Широта: от верхней к нижней (y увеличивается вниз)
    latitude = top_left_gps[0] - (y_norm * lat_range)
    # Долгота: от левой к правой (x увеличивается вправо)  
    longitude = top_left_gps[1] + (x_norm * lon_range)
    
    return (round(latitude, 8), round(longitude, 8))  # увеличим точность до 8 знаков

def process_map_with_yolo_tiled(model_path, image_path, output_json_path, output_viz_path, tile_size=640, overlap=64, conf=0.3):
    """Обработка карты YOLO с тайлингом и визуализацией"""
    
    # Извлекаем координаты из названия файла
    top_left_gps, bottom_right_gps = extract_coords_from_filename(os.path.basename(image_path))
    
    if not top_left_gps:
        return
    
    print(f"📍 Координаты из файла: С-З {top_left_gps}, Ю-В {bottom_right_gps}")
    
    # Загружаем модель и изображение
    print("🔄 Загружаю модель YOLO...")
    model = YOLO(model_path)
    print("✅ Модель загружена")
    
    print(f"🖼️ Загружаю изображение: {image_path}")
    original_image = cv2.imread(image_path)
    if original_image is None:
        print("❌ Не могу загрузить изображение")
        return
        
    h, w = original_image.shape[:2]
    print(f"📐 Размер изображения: {w}x{h}")
    
    # Создаем копию для визуализации
    visualization_image = original_image.copy()
    
    # Создаем данные для JSON
    centroids_data = {
        "metadata": {
            "source_image": image_path,
            "image_size": {"width": w, "height": h},
            "gps_bounds": {
                "top_left": list(top_left_gps),
                "bottom_right": list(bottom_right_gps)
            },
            "processing_date": datetime.now().isoformat(),
            "model_used": model_path,
            "processing_params": {
                "tile_size": tile_size,
                "overlap": overlap,
                "confidence_threshold": conf
            }
        },
        "objects": []
    }
    
    # ПРАВИЛЬНЫЙ РАСЧЕТ КОЛИЧЕСТВА ТАЙЛОВ
    x_tiles = 0
    y_tiles = 0
    tile_positions = []
    
    # Собираем все позиции тайлов
    for y in range(0, h, tile_size - overlap):
        for x in range(0, w, tile_size - overlap):
            x1 = x
            y1 = y
            x2 = min(x + tile_size, w)
            y2 = min(y + tile_size, h)
            
            # УБИРАЕМ УСЛОВИЕ ПРОПУСКА - обрабатываем ВСЕ тайлы
            # даже если они маленькие, чтобы не пропустить края
            tile_positions.append((x1, y1, x2, y2))
    
    total_tiles = len(tile_positions)
    
    print(f"🧩 Количество тайлов: {total_tiles}")
    print(f"🔲 Размер тайла: {tile_size}x{tile_size}")
    print(f"🔄 Перекрытие: {overlap} пикселей")
    
    processed_tiles = 0
    object_count = 0
    
    # Обрабатываем каждый тайл
    print("🔍 Начинаю обработку тайлов...")
    for x1, y1, x2, y2 in tqdm(tile_positions, desc="Обработка тайлов"):
        # Вырезаем тайл (даже если он маленький)
        tile = original_image[y1:y2, x1:x2]
        
        # Обрабатываем тайл моделью
        results = model(tile, conf=conf, verbose=False)
        
        # Обрабатываем результаты для этого тайла
        for r in results:
            if r.masks is not None and len(r.masks) > 0:
                for i, mask in enumerate(r.masks.data):
                    if i < len(r.boxes.cls):
                        class_id = int(r.boxes.cls[i])
                        confidence = float(r.boxes.conf[i])
                        
                        # Конвертируем маску
                        mask_np = mask.cpu().numpy()
                        mask_resized = cv2.resize(mask_np, (x2-x1, y2-y1))
                        mask_binary = (mask_resized > 0.5).astype(np.uint8) * 255
                        
                        # Вычисляем центроид
                        centroid = calculate_centroid(mask_binary)
                        
                        if centroid:
                            # Преобразуем в глобальные координаты
                            global_centroid_px = (x1 + centroid[0], y1 + centroid[1])
                            
                            # Конвертируем в GPS
                            gps_coords = pixel_to_gps(global_centroid_px, w, h, top_left_gps, bottom_right_gps)
                            
                            # Сохраняем данные
                            object_data = {
                                "class_id": class_id,
                                "confidence": confidence,
                                "gps_coordinates": {
                                    "latitude": gps_coords[0],
                                    "longitude": gps_coords[1]
                                }
                            }
                            centroids_data["objects"].append(object_data)
                            object_count += 1
            
            # ВИЗУАЛИЗАЦИЯ: рисуем полигоны на изображении
            if hasattr(r, 'plot') and r.boxes is not None:
                plotted_tile = r.plot()  # получаем тайл с нарисованными полигонами
                # Вставляем обработанный тайл в визуализацию
                visualization_image[y1:y2, x1:x2] = plotted_tile
        
        processed_tiles += 1
    
    # Сохраняем визуализацию с полигонами
    cv2.imwrite(output_viz_path, visualization_image)
    print(f"🖼️ Визуализация с полигонами сохранена: {output_viz_path}")
    
    # Сохраняем JSON
    with open(output_json_path, 'w', encoding='utf-8') as f:
        json.dump(centroids_data, f, ensure_ascii=False, indent=2)
    
    print(f"\n✅ Итоги обработки:")
    print(f"   Обработано тайлов: {processed_tiles}/{total_tiles}")
    print(f"   Обнаружено объектов: {object_count}")
    print(f"   JSON сохранен: {output_json_path}")
    print(f"   Визуализация сохранена: {output_viz_path}")
    
    # Статистика по классам
    if object_count > 0:
        class_counts = {}
        for obj in centroids_data["objects"]:
            class_id = obj["class_id"]
            class_counts[class_id] = class_counts.get(class_id, 0) + 1
        
        print("📊 Статистика по классам:")
        for class_id, count in class_counts.items():
            class_name = model.names[class_id]
            print(f"   - {class_name}: {count} объектов")
if __name__ == "__main__":
    # Настройки
    MODEL_PATH = '../runs/segment/yolov8n_gpu_simple_1/weights/best.pt'  # путь к твоей модели
    IMAGE_PATH = "output_img/map_55d948091_37d941703_to_55d967844_37d996474.jpg"  # карта из папки maps
    base_name = os.path.splitext(os.path.basename(IMAGE_PATH))[0].replace('map_', '')

# Создаем названия файлов с координатами
    OUTPUT_JSON = os.path.join("output_json", f"json_{base_name}.json")
    OUTPUT_POLYGONS = os.path.join("output_yolo_img", f"polygons_{base_name}.jpg")
    

     # Параметры тайлинга
    TILE_SIZE = 640
    OVERLAP = 10
    CONFIDENCE = 0.25
    
    print(f"📁 Выходные файлы:")
    print(f"   JSON: {OUTPUT_JSON}")
    print(f"   Polygons: {OUTPUT_POLYGONS}")
    
    print("🚀 Запуск обработки YOLO...")
    process_map_with_yolo_tiled(
        MODEL_PATH, 
        IMAGE_PATH, 
        OUTPUT_JSON,
        OUTPUT_POLYGONS,
        tile_size=TILE_SIZE,
        overlap=OVERLAP,
        conf=CONFIDENCE
    )

📁 Выходные файлы:
   JSON: output_json\json_55d948091_37d941703_to_55d967844_37d996474.json
   Polygons: output_yolo_img\polygons_55d948091_37d941703_to_55d967844_37d996474.jpg
🚀 Запуск обработки YOLO...
🔍 Извлеченные координаты из имени файла:
   Точка 1: 55.948091, 37.941703
   Точка 2: 55.967844, 37.996474
   Северо-запад (top_left): (55.967844, 37.941703)
   Юго-восток (bottom_right): (55.948091, 37.996474)
📍 Координаты из файла: С-З (55.967844, 37.941703), Ю-В (55.948091, 37.996474)
🔄 Загружаю модель YOLO...
✅ Модель загружена
🖼️ Загружаю изображение: output_img/map_55d948091_37d941703_to_55d967844_37d996474.jpg
📐 Размер изображения: 10496x6912
🧩 Количество тайлов: 187
🔲 Размер тайла: 640x640
🔄 Перекрытие: 10 пикселей
🔍 Начинаю обработку тайлов...


Обработка тайлов: 100%|██████████| 187/187 [00:06<00:00, 27.16it/s]


🖼️ Визуализация с полигонами сохранена: output_yolo_img\polygons_55d948091_37d941703_to_55d967844_37d996474.jpg

✅ Итоги обработки:
   Обработано тайлов: 187/187
   Обнаружено объектов: 936
   JSON сохранен: output_json\json_55d948091_37d941703_to_55d967844_37d996474.json
   Визуализация сохранена: output_yolo_img\polygons_55d948091_37d941703_to_55d967844_37d996474.jpg
📊 Статистика по классам:
   - building: 859 объектов
   - field: 26 объектов
   - lake: 15 объектов
   - zrail: 24 объектов
   - forest: 12 объектов


Висаулизация центров

In [3]:
import json
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import os

def visualize_polygons_from_json(json_path, output_image_path):
    """Визуализация всех полигонов из JSON"""
    
    # Загружаем данные
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    metadata = data['metadata']
    objects = data['objects']
    
    # Извлекаем границы
    gps_bounds = metadata['gps_bounds']
    top_left = gps_bounds['top_left']
    bottom_right = gps_bounds['bottom_right']
    
    print(f"📍 GPS границы: С-З {top_left}, Ю-В {bottom_right}")
    print(f"📊 Всего объектов: {len(objects)}")
    
    # Создаем карту
    fig, ax = plt.subplots(figsize=(16, 12))
    
    # Цвета для классов
    class_colors = {
        0: 'red',      # building
        1: 'green',    # field
        2: 'darkgreen', # forest
        3: 'blue',     # lake
        4: 'gray',     # road
        5: 'orange'    # zrail
    }
    
    class_names = {
        0: 'Building',
        1: 'Field',
        2: 'Forest',
        3: 'Lake',
        4: 'Road', 
        5: 'Railway'
    }
    
    # Собираем данные по классам
    class_data = {}
    for obj in objects:
        class_id = obj['class_id']
        if class_id not in class_data:
            class_data[class_id] = []
        class_data[class_id].append(obj)
    
    # Рисуем полигоны (точки с областями)
    for class_id, objects_list in class_data.items():
        if objects_list:
            lons = [obj['gps_coordinates']['longitude'] for obj in objects_list]
            lats = [obj['gps_coordinates']['latitude'] for obj in objects_list]
            confidences = [obj['confidence'] for obj in objects_list]
            
            color = class_colors.get(class_id, 'purple')
            
            # Рисуем точки с размерами по confidence
            sizes = [10 + conf * 50 for conf in confidences]  # размер зависит от confidence
            scatter = ax.scatter(lons, lats, 
                               c=color, 
                               s=sizes,
                               alpha=0.6,
                               edgecolors='black',
                               linewidth=0.5,
                               label=class_names.get(class_id, f'Class {class_id}'))
    
    # Настройки карты
    ax.set_xlabel('Долгота (Longitude)', fontsize=12)
    ax.set_ylabel('Широта (Latitude)', fontsize=12)
    ax.set_title('Визуализация всех полигонов объектов', fontsize=16, pad=20)
    
    # Сетка
    ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.5)
    
    # Легенда
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
    
    # Устанавливаем границы - ИСПРАВЛЕННАЯ ЧАСТЬ
    ax.set_xlim(top_left[1], bottom_right[1])  # долгота от западной к восточной
    ax.set_ylim(top_left[0], bottom_right[0])  # широта от северной к южной
    
    # Сохраняем
    plt.tight_layout()
    plt.savefig(output_image_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"✅ Визуализация полигонов сохранена: {output_image_path}")
    
    # Статистика
    print("📊 Статистика по классам:")
    for class_id, objects_list in class_data.items():
        class_name = class_names.get(class_id, f'Class {class_id}')
        print(f"   - {class_name}: {len(objects_list)} объектов")

if __name__ == "__main__":
    
    JSON_PATH = "output_json/json_55d948091_37d941703_to_55d967844_37d996474.json"
    base_name = os.path.splitext(os.path.basename(JSON_PATH))[0].replace('json_', '')
    OUTPUT_CENTROIDS = os.path.join("output_yolo_img", f"centroids_{base_name}.jpg")
    
    visualize_polygons_from_json(JSON_PATH, OUTPUT_CENTROIDS)

📍 GPS границы: С-З [55.948091, 37.941703], Ю-В [55.967844, 37.996474]
📊 Всего объектов: 936
✅ Визуализация полигонов сохранена: output_yolo_img\centroids_55d948091_37d941703_to_55d967844_37d996474.jpg
📊 Статистика по классам:
   - Building: 859 объектов
   - Field: 26 объектов
   - Lake: 15 объектов
   - Railway: 24 объектов
   - Forest: 12 объектов


In [13]:
import cv2
import numpy as np
import re
import os

def parse_coordinates(filename):
    """Парсинг координат из имени файла"""
    # Извлекаем только имя файла из пути
    basename = os.path.basename(filename)
    
    # Ищем координаты в формате map_ll_lat_ll_lon_to_ur_lat_ur_lon
    # Разрешаем символы d в числах (например, 55d948091)
    pattern = r'map_([\dd\.]+)_([\dd\.]+)_to_([\dd\.]+)_([\dd\.]+)'
    match = re.search(pattern, basename)
    if match:
        # Убираем символы 'd' из координат и преобразуем в float
        ll_lat = float(match.group(1).replace('d', ''))
        ll_lon = float(match.group(2).replace('d', ''))
        ur_lat = float(match.group(3).replace('d', ''))
        ur_lon = float(match.group(4).replace('d', ''))
        
        return {
            'll_lat': ll_lat,
            'll_lon': ll_lon,
            'ur_lat': ur_lat,
            'ur_lon': ur_lon
        }
    else:
        print(f"Не удалось распарсить координаты из: {basename}")
        print(f"Найденные группы: {match.groups() if match else 'нет совпадений'}")
        return None

def latlon_to_pixel(lat, lon, coords, image_width, image_height):
    """Конвертация географических координат в пиксели изображения"""
    lat_ratio = (coords['ur_lat'] - lat) / (coords['ur_lat'] - coords['ll_lat'])
    lon_ratio = (lon - coords['ll_lon']) / (coords['ur_lon'] - coords['ll_lon'])
    
    x = int(lon_ratio * image_width)
    y = int(lat_ratio * image_height)
    
    return x, y

def smoothstep(edge0, edge1, x):
    """Плавная функция интерполяции"""
    x = np.clip((x - edge0) / (edge1 - edge0), 0.0, 1.0)
    return x * x * (3 - 2 * x)

def create_route_visualization(main_image, coords, waypoints, output_path="route_visualization.jpg"):
    """Создает визуализацию маршрута на карте"""
    route_viz = main_image.copy()
    h, w = route_viz.shape[:2]
    
    # Рисуем маршрут
    route_points = []
    for lat, lon in waypoints:
        x, y = latlon_to_pixel(lat, lon, coords, w, h)
        route_points.append((x, y))
    
    # Рисуем линию маршрута
    for i in range(len(route_points) - 1):
        cv2.line(route_viz, route_points[i], route_points[i+1], (0, 255, 0), 3)
    
    # Рисуем точки маршрута
    for i, (x, y) in enumerate(route_points):
        color = (0, 0, 255) if i == 0 else (255, 0, 0) if i == len(route_points)-1 else (0, 255, 255)
        cv2.circle(route_viz, (x, y), 8, color, -1)
        cv2.putText(route_viz, f"{i+1}", (x-5, y-10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
    
    # Добавляем легенду
    cv2.putText(route_viz, "Drone Flight Route", (20, 30), 
               cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    cv2.putText(route_viz, "Start (Red)", (20, 60), 
               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    cv2.putText(route_viz, "Waypoints (Yellow)", (20, 85), 
               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
    cv2.putText(route_viz, "End (Blue)", (20, 110), 
               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
    
    cv2.imwrite(output_path, route_viz)
    print(f"Визуализация маршрута сохранена: {output_path}")
    return route_viz

def create_drone_animation_with_gps(input_image, output_video, duration_seconds=30, 
                                   fps=60, output_size=(640, 640), altitude=100):
    """
    Создает анимацию полета дрона с привязкой к GPS координатам
    """
    
    # Парсим координаты из имени файла
    coords = parse_coordinates(input_image)
    if not coords:
        print("Не удалось распарсить координаты из имени файла")
        return
    
    print(f"Координаты карты:")
    print(f"Нижний левый: ({coords['ll_lat']:.6f}, {coords['ll_lon']:.6f})")
    print(f"Верхний правый: ({coords['ur_lat']:.6f}, {coords['ur_lon']:.6f})")
    
    # Загружаем основное изображение
    main_image = cv2.imread(input_image)
    if main_image is None:
        print("Не удалось загрузить изображение")
        return
    
    h, w = main_image.shape[:2]
    
    # Генерация более плавного маршрута с большим количеством точек
    waypoints = [
        # Старт - нижний левый угол (медленный взлет)
        (coords['ll_lat'] + 0.05 * (coords['ur_lat'] - coords['ll_lat']), 
         coords['ll_lon'] + 0.05 * (coords['ur_lon'] - coords['ll_lon'])),
        
        # Плавный подъем
        (coords['ll_lat'] + 0.15 * (coords['ur_lat'] - coords['ll_lat']), 
         coords['ll_lon'] + 0.1 * (coords['ur_lon'] - coords['ll_lon'])),
        
        # Первая точка - слева посередине
        (coords['ll_lat'] + 0.25 * (coords['ur_lat'] - coords['ll_lat']), 
         coords['ll_lon'] + 0.15 * (coords['ur_lon'] - coords['ll_lon'])),
        
        # Плавный поворот
        (coords['ll_lat'] + 0.35 * (coords['ur_lat'] - coords['ll_lat']), 
         coords['ll_lon'] + 0.25 * (coords['ur_lon'] - coords['ll_lon'])),
        
        # Центр
        (coords['ll_lat'] + 0.45 * (coords['ur_lat'] - coords['ll_lat']), 
         coords['ll_lon'] + 0.4 * (coords['ur_lon'] - coords['ll_lon'])),
        
        # Вторая точка
        (coords['ll_lat'] + 0.55 * (coords['ur_lat'] - coords['ll_lat']), 
         coords['ll_lon'] + 0.55 * (coords['ur_lon'] - coords['ll_lon'])),
        
        # Плавный поворот к финишу
        (coords['ll_lat'] + 0.65 * (coords['ur_lat'] - coords['ll_lat']), 
         coords['ll_lon'] + 0.7 * (coords['ur_lon'] - coords['ll_lon'])),
        
        # Подготовка к посадке
        (coords['ll_lat'] + 0.75 * (coords['ur_lat'] - coords['ll_lat']), 
         coords['ll_lon'] + 0.8 * (coords['ur_lon'] - coords['ll_lon'])),
        
        # Финиш - верхний правый угол (медленная посадка)
        (coords['ur_lat'] - 0.05 * (coords['ur_lat'] - coords['ll_lat']), 
         coords['ur_lon'] - 0.05 * (coords['ur_lon'] - coords['ll_lon']))
    ]
    
    # Создаем визуализацию маршрута
    route_viz = create_route_visualization(main_image, coords, waypoints, "flight_route.jpg")
    
    # Создаем видео writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video, fourcc, fps, output_size)
    
    total_frames = int(fps * duration_seconds)
    
    print("Генерация плавной анимации с GPS привязкой...")
    print(f"Маршрут через {len(waypoints)} точек")
    print(f"Длительность: {duration_seconds} сек, Кадров: {total_frames}")
    
    # Предварительно вычисляем все позиции для максимальной плавности
    positions = []
    for frame_num in range(total_frames):
        progress = frame_num / total_frames
        
        # Очень плавная функция прогресса с замедлением в начале и конце
        eased_progress = smoothstep(0, 1, progress)
        
        # Определяем текущий сегмент маршрута
        segment_progress = eased_progress * (len(waypoints) - 1)
        segment_index = min(int(segment_progress), len(waypoints) - 2)
        local_progress = segment_progress - segment_index
        
        # Супер-плавная интерполяция между точками
        super_smooth_progress = 0.5 - 0.5 * np.cos(local_progress * np.pi)
        
        # Текущая позиция в GPS координатах
        start_lat, start_lon = waypoints[segment_index]
        end_lat, end_lon = waypoints[segment_index + 1]
        
        current_lat = start_lat + (end_lat - start_lat) * super_smooth_progress
        current_lon = start_lon + (end_lon - start_lon) * super_smooth_progress
        
        # Конвертируем в пиксели
        x_center, y_center = latlon_to_pixel(current_lat, current_lon, coords, w, h)
        
        # Плавное изменение высоты (масштаба) с замедлением в начале и конце
        start_scale = 0.3   # Очень высоко в начале
        mid_scale = 0.6     # Средняя высота
        end_scale = 0.4     # Немного опускаемся в конце
        
        if eased_progress < 0.3:  # Медленный взлет
            height_progress = eased_progress / 0.3
            current_scale = start_scale + (mid_scale - start_scale) * smoothstep(0, 1, height_progress)
        elif eased_progress < 0.7:  # Плавный полет
            current_scale = mid_scale
        else:  # Медленная посадка
            height_progress = (eased_progress - 0.7) / 0.3
            current_scale = mid_scale + (end_scale - mid_scale) * smoothstep(0, 1, height_progress)
        
        positions.append((x_center, y_center, current_lat, current_lon, current_scale))
    
    # Генерируем кадры видео
    print("Генерация кадров...")
    for frame_num in range(total_frames):
        x_center, y_center, current_lat, current_lon, current_scale = positions[frame_num]
        progress = frame_num / total_frames
        
        # Вычисляем размер для текущего кадра
        crop_w = int(output_size[0] / current_scale)
        crop_h = int(output_size[1] / current_scale)
        
        # Вычисляем область обрезки
        x1 = max(0, x_center - crop_w // 2)
        y1 = max(0, y_center - crop_h // 2)
        x2 = min(w, x_center + crop_w // 2)
        y2 = min(h, y_center + crop_h // 2)
        
        # Вырезаем и масштабируем
        if x2 > x1 and y2 > y1:
            cropped = main_image[y1:y2, x1:x2]
            
            # Плавное масштабирование с интерполяцией
            resized = cv2.resize(cropped, output_size, interpolation=cv2.INTER_LANCZOS4)
            
            # Добавляем информацию о полете
            cv2.putText(resized, f"Altitude: {int(altitude * current_scale/0.3)}m", (10, 30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            cv2.putText(resized, f"GPS: {current_lat:.6f}, {current_lon:.6f}", (10, 60), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)
            cv2.putText(resized, f"Speed: 4 m/s", (10, 85), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            cv2.putText(resized, f"Progress: {progress*100:.1f}%", (10, 110), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            
            # Добавляем индикатор текущей позиции на мини-карте
            mini_map_size = 120
            mini_map = cv2.resize(route_viz, (mini_map_size, mini_map_size), interpolation=cv2.INTER_LANCZOS4)
            
            # Отмечаем текущую позицию на мини-карте
            mini_x = int((x_center / w) * mini_map_size)
            mini_y = int((y_center / h) * mini_map_size)
            cv2.circle(mini_map, (mini_x, mini_y), 4, (0, 0, 255), -1)
            
            # Рисуем траекторию пройденного пути на мини-карте
            for i in range(min(frame_num // 10, len(positions) // 10)):
                idx = i * 10
                if idx < len(positions):
                    px, py, _, _, _ = positions[idx]
                    trail_x = int((px / w) * mini_map_size)
                    trail_y = int((py / h) * mini_map_size)
                    cv2.circle(mini_map, (trail_x, trail_y), 1, (255, 255, 0), -1)
            
            # Вставляем мини-карту в основной кадр
            resized[10:10+mini_map_size, output_size[0]-10-mini_map_size:output_size[0]-10] = mini_map
            
            # Рамка вокруг мини-карты
            cv2.rectangle(resized, 
                         (output_size[0]-10-mini_map_size, 10),
                         (output_size[0]-10, 10+mini_map_size),
                         (255, 255, 255), 2)
            
            out.write(resized)
        
        # Прогресс
        if frame_num % (fps * 5) == 0:  # Сообщение каждые 5 секунд
            print(f"Обработано: {progress*100:.1f}%")
    
    out.release()
    print(f"✅ Видео сохранено: {output_video}")
    print(f"📊 Длительность: {duration_seconds} сек")
    print(f"🎞️ Кадров: {total_frames}")
    print(f"🔄 FPS: {fps}")
    print(f"📏 Размер: {output_size[0]}x{output_size[1]}")
    print(f"🛩️ Высота: {altitude} м")

# Использование
if __name__ == "__main__":
    # Укажите путь к вашему файлу карты
    input_map = "output_img/map_55d948091_37d941703_to_55d967844_37d996474.jpg"
    
    if os.path.exists(input_map):
        create_drone_animation_with_gps(
            input_image=input_map,
            output_video="drone_flight_smooth.mp4",
            duration_seconds=30,  # Увеличил длительность в 2 раза
            fps=60,
            output_size=(640, 640),
            altitude=100
        )
    else:
        print(f"Файл {input_map} не найден!")
        print("Убедитесь, что файл карты существует в текущей директории")

Координаты карты:
Нижний левый: (55948091.000000, 37941703.000000)
Верхний правый: (55967844.000000, 37996474.000000)
Визуализация маршрута сохранена: flight_route.jpg
Генерация плавной анимации с GPS привязкой...
Маршрут через 9 точек
Длительность: 30 сек, Кадров: 1800
Генерация кадров...
Обработано: 0.0%
Обработано: 16.7%


KeyboardInterrupt: 

In [ ]:
import cv2
from ultralytics import YOLO

# Загружаем вашу модель YOLO
model = YOLO('../runs/segment/yolov8n_gpu_updgrade_1/weights/best.pt')

# Открываем видео с дроном
cap = cv2.VideoCapture('drone_flight_smooth.mp4')

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Детекция на кадре - ИСПРАВЛЕНО: используем predict()
    results = model.predict(frame, verbose=False)
    
    # Рисуем результаты
    detected_frame = results[0].plot()
    
    # Показываем результат
    cv2.imshow('YOLO Detection', detected_frame)
    
    # Выход по 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
